In [ ]:
!pip install -qU transformers datasets evaluate accelerate tensorflow

In [ ]:
import collections

import numpy as np
import tensorflow as tf
from transformers import AutoTokenizer, TFAutoModelForQuestionAnswering
from transformers import DefaultDataCollator
from transformers import create_optimizer
from datasets import load_dataset, load_metric
from tqdm.auto import tqdm
import evaluate

In [ ]:
raw_datasets = load_dataset("squad")
metric = evaluate.load("squad")
tokenizer = AutoTokenizer.from_pretrained('distilbert-base-cased')

In [ ]:
max_length = 384
stride = 128

def handle_data_for_train(examples):
    question_stripped = [q.strip() for q in examples["question"]]
    tokenized_inputs = tokenizer(
        question_stripped,
        examples["context"],
        return_offsets_mapping=True,
        truncation="only_second",
        return_overflowing_tokens=True,
        max_length=max_length,
        stride=stride,
        padding="max_length"
    )

    offset_mapping = tokenized_inputs.pop("offset_mapping")
    sample_map = tokenized_inputs.pop("overflow_to_sample_mapping")
    answers = examples["answers"]
    start_positions = []
    end_positions = []

    for i, offsets in enumerate(offset_mapping):
        sample_idx = sample_map[i]
        answer = answers[sample_idx]
        start_char = answer["answer_start"][0]
        end_char = answer["answer_start"][0] + len(answer["text"][0])
        sequence_ids = tokenized_inputs.sequence_ids(i)

        idx = 0
        while sequence_ids[idx] != 1:
            idx += 1
        context_start = idx
        while sequence_ids[idx] == 1:
            idx += 1
        context_end = idx - 1

        if offsets[context_start][0] > start_char or offsets[context_end][1] < end_char:
            start_positions.append(0)
            end_positions.append(0)
        else:
            idx = context_start
            while idx <= context_end and offsets[idx][0] <= start_char:
                idx += 1
            start_positions.append(idx - 1)

            idx = context_end
            while idx >= context_start and offsets[idx][1] >= end_char:
                idx -= 1
            end_positions.append(idx + 1)

    tokenized_inputs["start_positions"] = start_positions
    tokenized_inputs["end_positions"] = end_positions
    return tokenized_inputs

In [ ]:
train_dataset = raw_datasets["train"].map(
    handle_data_for_train,
    batched=True,
    remove_columns=raw_datasets["train"].column_names,
)

In [ ]:
def handle_data_for_valid(examples):
    questions_stripped = [q.strip() for q in examples["question"]]
    tokenized_inputs = tokenizer(
        questions_stripped,
        examples["context"],
        max_length=max_length,
        stride=stride,
        return_offsets_mapping=True,
        padding="max_length",
        return_overflowing_tokens=True,
        truncation="only_second",
    )

    sample_map = tokenized_inputs.pop("overflow_to_sample_mapping")
    example_ids = []

    for i in range(len(tokenized_inputs["input_ids"])):
        sample_idx = sample_map[i]
        example_ids.append(examples["id"][sample_idx])

        sequence_ids = tokenized_inputs.sequence_ids(i)
        offsets = tokenized_inputs["offset_mapping"][i]
        tokenized_inputs["offset_mapping"][i] = [
            o if sequence_ids[k] == 1 else None for k, o in enumerate(offsets)
        ]

    tokenized_inputs["example_id"] = example_ids
    return tokenized_inputs


In [ ]:
validation_dataset = raw_datasets["validation"].map(
    handle_data_for_valid,
    batched=True,
    remove_columns=raw_datasets["validation"].column_names,
)

In [ ]:
trained_checkpoint = "distilbert-base-cased"
tokenizer = AutoTokenizer.from_pretrained(trained_checkpoint)
model = TFAutoModelForQuestionAnswering.from_pretrained(trained_checkpoint)

In [ ]:
def compute_metrics(predictions_start_logits, predictions_end_logits, input_features, evaluation_examples,
                    n_best=5, max_ans_len=30):
    example_to_features = collections.defaultdict(list)
    for idx, feature in enumerate(input_features):
        example_to_features[feature["example_id"]].append(idx)

    predicted_answers = []
    for example in tqdm(evaluation_examples):
        example_id = example["id"]
        context = example["context"]
        answer_spans = []

        for feature_index in example_to_features[example_id]:
            start_logit = predictions_start_logits[feature_index]
            end_logit = predictions_end_logits[feature_index]
            offsets = input_features[feature_index]["offset_mapping"]

            start_indexes = np.argsort(start_logit)[-1 : -n_best - 1 : -1].tolist()
            end_indexes = np.argsort(end_logit)[-1 : -n_best - 1 : -1].tolist()
            for start_index in start_indexes:
                for end_index in end_indexes:

                    if (
                        end_index < start_index
                        or end_index - start_index + 1 > max_ans_len
                        or offsets[start_index] is None or offsets[end_index] is None
                    ):
                        continue

                    answer_span = {
                        "text": context[offsets[start_index][0] : offsets[end_index][1]],
                        "logit_score": start_logit[start_index] + end_logit[end_index],
                    }
                    answer_spans.append(answer_span)

        if len(answer_spans) > 0:
            best_answer_span = max(answer_spans, key=lambda x: x["logit_score"])
            predicted_answers.append(
                {"id": example_id, "prediction_text": best_answer_span["text"]}
            )
        else:
            predicted_answers.append({"id": example_id, "prediction_text": ""})

    ground_truth_answers = [{"id": ex["id"], "answers": ex["answers"]} for ex in evaluation_examples]
    return metric.compute(predictions=predicted_answers, references=ground_truth_answers), predicted_answers, ground_truth_answers


In [ ]:
data_collator = DefaultDataCollator(return_tensors="tf")

model = TFAutoModelForQuestionAnswering.from_pretrained(trained_checkpoint)

tf_train_dataset = model.prepare_tf_dataset(
    train_dataset,
    collate_fn=data_collator,
    shuffle=True,
    batch_size=8,
)
tf_eval_dataset = model.prepare_tf_dataset(
    validation_dataset,
    collate_fn=data_collator,
    shuffle=False,
    batch_size=16,
)

num_train_epochs = 6
num_train_steps = len(tf_train_dataset) * num_train_epochs

optimizer, schedule = create_optimizer(
    init_lr=2e-5,
    num_warmup_steps=0,
    num_train_steps=num_train_steps,
    weight_decay_rate=0.01,
)
model.compile(optimizer=optimizer)

tf.keras.mixed_precision.set_global_policy("mixed_float16")

In [ ]:
model.fit(tf_train_dataset, epochs=num_train_epochs)

In [ ]:
predictions = model.predict(tf_eval_dataset)
metrics, predicted_answers, theoretical_answers = compute_metrics(
    predictions["start_logits"],
    predictions["end_logits"],
    validation_dataset,
    raw_datasets["validation"],
)
metrics

In [ ]:
model.save_pretrained("model")